In [1]:
import pandas as pd
import numpy as np

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet

from sklearn.metrics import mean_absolute_error, mean_squared_error
from google.colab import files

import warnings
warnings.filterwarnings("ignore")


In [2]:
retail = pd.read_csv("/content/retail_price_data.csv")
wholesale = pd.read_csv("/content/wholesale_price_data (2).csv")

retail = retail.rename(columns={"States/UTs": "state", "Date": "date"})
wholesale = wholesale.rename(columns={"States/UTs": "state", "Date": "date"})

retail["date"] = pd.to_datetime(retail["date"], errors="coerce")
wholesale["date"] = pd.to_datetime(wholesale["date"], errors="coerce")


In [3]:
retail_long = retail.melt(
    id_vars=["state", "date"],
    var_name="commodity",
    value_name="price"
)
retail_long["market"] = "Retail"

wholesale_long = wholesale.melt(
    id_vars=["state", "date"],
    var_name="commodity",
    value_name="price"
)
wholesale_long["market"] = "Wholesale"

data = pd.concat([retail_long, wholesale_long], ignore_index=True)
data["price"] = pd.to_numeric(data["price"], errors="coerce")


In [4]:
BUCKETS = {
    "Bucket1": {
        "crops": ["Rice", "Atta (Wheat)", "Milk", "Tea Loose"],
        "model": "ARIMA"
    },
    "Bucket2": {
        "crops": [
            "Wheat","Sugar","Mustard Oil (Packed)",
            "Sunflower Oil (Packed)","Palm Oil (Packed)",
            "Tomato","Moong Dal","Masoor Dal"
        ],
        "model": "SARIMA"
    },
    "Bucket3": {
        "crops": ["Onion","Potato","Salt Pack"],
        "model": "PROPHET"
    },
    "Bucket4": {
        "crops": [
            "Gram Dal","Gur","Groundnut Oil (Packed)",
            "Urad Dal","Tur/Arhar Dal","Soya Oil (Packed)"
        ],
        "model": "BEST"
    }
}


In [5]:
def prepare_series(df, state, crop, market):
    ts = df[
        (df["state"] == state) &
        (df["commodity"] == crop) &
        (df["market"] == market)
    ].sort_values("date")

    ts = ts.set_index("date")["price"]
    ts = ts.dropna()
    ts = ts.asfreq("D").ffill()
    return ts


In [6]:
def run_arima(train, steps):
    model = ARIMA(train, order=(1,1,1)).fit()
    return model.forecast(steps)

def run_sarima(train, steps):
    model = SARIMAX(
        train,
        order=(1,1,1),
        seasonal_order=(1,0,1,12),
        enforce_stationarity=False,
        enforce_invertibility=False
    ).fit(disp=False)
    return model.forecast(steps)

def run_prophet(train, steps):
    df = train.reset_index()
    df.columns = ["ds","y"]

    model = Prophet(yearly_seasonality=True)
    model.fit(df)

    future = model.make_future_dataframe(periods=steps)
    pred = model.predict(future)

    return pred.tail(steps)["yhat"].values


In [7]:
def evaluate(train, test, model_type):
    if model_type == "ARIMA":
        pred = run_arima(train, len(test))
    elif model_type == "SARIMA":
        pred = run_sarima(train, len(test))
    else:
        pred = run_prophet(train, len(test))

    return {
        "RMSE": np.sqrt(mean_squared_error(test, pred)),
        "MAE": mean_absolute_error(test, pred)
    }


In [8]:
forecast_rows = []
accuracy_rows = []

for bucket, info in BUCKETS.items():
    for crop in info["crops"]:

        subset = data[data["commodity"] == crop]

        for (state, market), _ in subset.groupby(["state","market"]):

            ts = prepare_series(data, state, crop, market)

            if len(ts) < 150:
                continue

            train, test = ts[:-30], ts[-30:]

            # Model choice
            if info["model"] == "BEST":
                scores = {}
                for m in ["ARIMA","SARIMA","PROPHET"]:
                    try:
                        scores[m] = evaluate(train, test, m)["RMSE"]
                    except:
                        scores[m] = np.inf
                model_type = min(scores, key=scores.get)
            else:
                model_type = info["model"]

            acc = evaluate(train, test, model_type)

            # Final forecast
            if model_type == "ARIMA":
                forecast = run_arima(ts, 30)
            elif model_type == "SARIMA":
                forecast = run_sarima(ts, 30)
            else:
                forecast = run_prophet(ts, 30)

            for d, p in zip(
                pd.date_range(ts.index[-1] + pd.Timedelta(days=1), periods=30),
                forecast
            ):
                forecast_rows.append({
                    "bucket": bucket,
                    "state": state,
                    "commodity": crop,
                    "market": market,
                    "date": d,
                    "forecast_price": p,
                    "model_used": model_type
                })

            accuracy_rows.append({
                "bucket": bucket,
                "state": state,
                "commodity": crop,
                "market": market,
                "model_used": model_type,
                **acc
            })


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
forecast_df = pd.DataFrame(forecast_rows)
accuracy_df = pd.DataFrame(accuracy_rows)

forecast_df.to_csv("all_buckets_forecasts.csv", index=False)
accuracy_df.to_csv("all_buckets_accuracy.csv", index=False)

files.download("all_buckets_forecasts.csv")
files.download("all_buckets_accuracy.csv")
